<a href="https://colab.research.google.com/github/josephmailil1/Speciale---Stibo/blob/main/LLM_Flow_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_Key') #use the name of your colab key

In [3]:
import pandas as pd

shap = pd.read_csv("shap.csv")

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, openai_api_key=openai_api_key)

In [5]:
pip install ucimlrepo

In [6]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features # Importing all features
y = adult.data.targets # Importing target varialbes

In [7]:
# Recoding to match the same
y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})

<ipython-input-7-65ecf31535d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})


In [8]:
data =  pd.concat([X, y], axis=1)

In [9]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

tool = PythonAstREPLTool(locals={"data": data, "shap": shap})
tool.invoke("data['age'].mean()")

38.64358543876172

In [11]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
llm_with_tools.invoke(
    "I have a dataframe 'data' and want to know the correlation between the 'age' and 'capital-gain' columns"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ataHE9pHtAzF3Rdmejyhuguw', 'function': {'arguments': '{"query":"import pandas as pd\\ndata = pd.DataFrame({\\n    \'age\': [25, 30, 35, 40, 45],\\n    \'capital-gain\': [5000, 15000, 25000, 35000, 45000]\\n})\\ndata[\'age\'].corr(data[\'capital-gain\'])"}', 'name': 'python_repl_ast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 126, 'total_tokens': 203}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_ea6eb70039', 'finish_reason': 'stop', 'logprobs': None}, id='run-37feed3c-0539-45df-9321-b6fc0e29363e-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "import pandas as pd\ndata = pd.DataFrame({\n    'age': [25, 30, 35, 40, 45],\n    'capital-gain': [5000, 15000, 25000, 35000, 45000]\n})\ndata['age'].corr(data['capital-gain'])"}, 'id': 'call_ataHE9pHtAzF3Rdmejyhuguw'}])

In [12]:
from langchain.output_parsers import JsonOutputToolsParser

In [13]:
parser = JsonOutputToolsParser(tool = tool)

In [14]:
system = f"""You have access to a pandas dataframe `data`. \
Here is the output of `data.head().to_markdown()`:

```
{data.head().to_markdown()}
```
You are also given the pandas dateframe shap. Containing shap values for the test set obsevations.
The shap obsevation shows which values are important for the final prediction
```
{shap.head().to_markdown()}
```
Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

In [15]:
chain = prompt | llm_with_tools | parser  # noqa
chain.invoke({"question": "What is the most important feature value for the first obsevation?'"})

[{'args': {'query': 'shap.iloc[0].idxmax()'}, 'type': 'python_repl_ast'}]

In [16]:
data['age'].corr(data['capital-gain'])

0.07722902236438102

In [33]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe `data`. \
Here is the output of `data.head().to_markdown()`:

```
{data.head().to_markdown()}
```
You are also given the pandas dateframe shap. Containing shap values for the test set obsevations.
The shap obsevation shows which values are important for the final prediction
```
{shap.head().to_markdown()}
```

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas.
Respond directly to the question once you have enough information to answer it."""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser )
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["response"])
)

In [60]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to two pandas DataFrames: `data` and `shap`.

`data` contains the dataset you're working with. Here is a preview of the first few rows:
{data.head().to_markdown()}

`shap` contains SHAP (SHapley Additive exPlanations) values for the test set observations. SHAP values represent the impact of each feature on the model's output. Here is a preview of the first few rows:
{shap.head().to_markdown()}

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas.
Respond directly to the question once you have enough information to answer it."""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser )
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["response"])
)

In [61]:
from langchain.memory import ConversationBufferMemory
# This is do give the bot memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [62]:
def call_chain(user_question):
    answer = chain.invoke({"question": user_question})
    return answer['response']

# Interface - Gradio

In [23]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.2 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=1a66d1f47693c03e77d7b5ce3723c0f56a38ea258a04697a7b06f772a3a0b257
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: typer


In [63]:
import gradio as gr

In [64]:
demo = gr.Interface(
    fn=call_chain,
    inputs=["text"],
    outputs=["text"]
)

In [65]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6a4e7306f989aa41c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [43]:
shap.loc[0, ['marital-status_Married-civ-spouse', 'education-num', 'occupation_Other-service', 'sex_Male']]

marital-status_Married-civ-spouse    0.037621
education-num                        0.053867
occupation_Other-service             0.000020
sex_Male                             0.004897
Name: 0, dtype: float64

In [44]:
shap.loc[0].nlargest(5)

Income                               0.247534
education-num                        0.053867
marital-status_Married-civ-spouse    0.037621
education_Bachelors                  0.016926
relationship_Not-in-family           0.011256
Name: 0, dtype: float64

In [56]:
important_features = shap.iloc[0].sort_values(ascending=False)
print(important_features)

Income                               0.247534
education-num                        0.053867
marital-status_Married-civ-spouse    0.037621
education_Bachelors                  0.016926
relationship_Not-in-family           0.011256
                                       ...   
occupation_Prof-specialty           -0.001490
capital-loss                        -0.001522
occupation_Exec-managerial          -0.003488
hours-per-week                      -0.004886
capital-gain                        -0.007084
Name: 0, Length: 97, dtype: float64


In [67]:
shap['marital-status_Married-civ-spouse'].mean()

0.0008067107222777425